**月報**https://mops.twse.com.tw/mops/web/t21sc04_ifrs

In [1]:
from excellib import set_excel_color_daily
from stocklib import get_stock_price, get_OTC_price, get_otc, get_investor, get_ratio
from stocklib import cal_bar, cal_KD, check_10days, check_10days_otc, get_trust_today
from stocklib import convert_num
from stock_plot import plot_K_chart


from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

In [2]:
import requests
import shutil
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import yfinance as yf
import json
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
from tqdm import tqdm
from glob import glob
%matplotlib inline

# ua = UserAgent()
root = '../'
path_trust = root + 'trust/'
path_check = root + 'daily_check/'
path_investor = root + 'investor/'
path_flow = root + 'flow/'

# 取得今天日期

In [3]:
d_today = date.today() #- timedelta(days=1)

today_tw = d_today - relativedelta(years=1911)
this_year = d_today.year
this_quarter = get_this_quarter()
n = 1
if d_today.weekday() == 0:
    n = 3
d_yesterday = d_today - timedelta(days=n)
s_yesterday = (d_today - timedelta(days=n)).strftime("%Y%m%d")
s_today_md = (d_today).strftime("%m/%d")
s_yesterday_md = (d_today - timedelta(days=n)).strftime("%m/%d")
s_today = d_today.strftime("%Y%m%d")

s_today_tw = today_tw.strftime("%Y/%m/%d").lstrip('0')
s_thismonth_tw = today_tw.strftime("%Y/%m").lstrip('0')
print('今天:', d_today, '昨天:', s_yesterday)
# 台灣證券交易所

今天: 2023-02-08 昨天: 20230207


In [4]:
df_yesterday = get_stock_price(d_yesterday)

df_today = get_stock_price(d_today)
df_today.head(2)

取得今日上市股票資訊: 20230207
取得今日上市股票資訊: 20230208


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,9239,120.1,120.95,120.0,120.85
0051,元大中型100,54,55.2,55.60,55.2,55.40


In [5]:
df_investor = get_investor(d_today)

lst_investor = list(df_investor[ (df_investor['investor']>0) & (df_investor['trust']>0)].index)
print('外資, 投信同步買超筆數：', len(lst_investor))
print(list(lst_investor))
df_investor.head(2)

外資, 投信同步買超筆數： 11665
外資, 投信同步買超筆數： 34
['3481', '2409', '2303', '2330', '3034', '0050', '4919', '2353', '2634', '2454', '2379', '3704', '6533', '2301', '3017', '2376', '6719', '1476', '2885', '6285', '2327', '4770', '2356', '6515', '6781', '2748', '3661', '4961', '4904', '2404', '1590', '6830', '6592A', '6282']


,investor,trust,dealer_T,dealer_F
id,,,,
2607,81004,0,-5,-1
3481,43375,1165,7533,3447


In [6]:
lst_investor_trust = list(df_investor[ (df_investor['investor']>100) | (df_investor['trust']>100)].index)
print('外資 or 投信買超筆數：', len(lst_investor_trust))

外資 or 投信買超筆數： 234


# 取得交易價量前30名

In [7]:
df_amt = df_today.copy()
df_amt['value_amt'] = df_amt['amount'] * df_amt['close'] # 成交量 * 收盤價(億)
df_amt['value_amt'] //= 100000
df_amt.sort_values('value_amt', ascending=False, inplace=True)
df_amt = df_amt[df_amt['close'] > df_amt['open']]
lst_amt = list(df_amt.head(30).index)
print('today:', lst_amt)

path_amt30 = './amt30.txt'
with open(path_amt30, 'r') as f:
    lst_amt30_yesterday = f.read().split('\n')
print('yesterday:', lst_amt30_yesterday)

with open(path_amt30, 'w') as f:
    for stock_id in lst_amt:
        f.write(stock_id + "\n")
lst_amt30_new = [i for i in lst_amt if i not in lst_amt30_yesterday]
print('\nnew:', lst_amt30_new)
df_amt = df_amt.head(30)
df_amt_new = df_amt.reset_index()
df_amt_new[df_amt_new['id'].isin(lst_amt30_new)]

today: ['2330', '3443', '3034', '3661', '3035', '2454', '1795', '2303', '4763', '6533', '6531', '8996', '6235', '4919', '3481', '2379', '6781', '6719', '2049', '6415', '2409', '3454', '1760', '5269', '6515', '2425', '3711', '0050', '8016', '00878']
yesterday: ['1795', '3443', '4763', '3661', '3034', '3037', '2303', '3035', '6282', '6235', '2345', '3454', '1752', '4919', '2201', '8046', '6415', '3008', '2376', '6533', '2609', '3481', '2748', '1783', '5258', '2409', '1513', '6515', '2615', '2049', '']

new: ['2330', '2454', '6531', '8996', '2379', '6781', '6719', '1760', '5269', '2425', '3711', '0050', '8016', '00878']


,id,name,amount,open,high,low,close,value_amt
0,2330,台積電,37633,538.00,540.00,534.0,540.00,203.0
5,2454,聯發科,7991,726.00,747.00,725.0,744.00,59.0
10,6531,愛普*,12757,248.50,268.00,246.0,268.00,34.0
11,8996,高力,14411,218.00,225.00,218.0,219.00,31.0
15,2379,瑞昱,7063,340.00,364.50,339.0,364.50,25.0
16,6781,AES-KY,2702,789.00,854.00,787.0,854.00,23.0
17,6719,力智,6509,312.00,330.50,311.0,330.50,21.0
22,1760,寶齡富錦,8307,176.00,180.50,174.5,179.50,14.0
23,5269,祥碩,1430,955.00,1010.00,948.0,1010.00,14.0
25,2425,承啟,32810,35.50,37.00,34.3,36.75,12.0


# 昨天外資、投信同步買超，檢查隔天是否有上漲

In [8]:
import yfinance as yf
def cal_mean_20(stock_id):
    data_yf = yf.Ticker(stock_id+'.TW')
    df = data_yf.history(period="1mo")
    df.sort_index(ascending=False, inplace=True)
    
    price_20 = round(np.mean(df['Close'][:20]),2)
    return price_20

In [9]:
# path_track_yesterday = path_investor + 'investors_2days_{}.xlsx'.format(s_yesterday)
# # shutil.copy2(file_track, file_track.replace('.xlsx', '_backup.xlsx')) 

# df_track_yes = pd.read_excel(path_track_yesterday, index_col=None, engine='openpyxl')
# df_track_yes['id'] = df_track_yes['id'].astype(str)
# lst_track_yes = list(set(df_track_yes['id']))
# print('昨天外資投信同步買超筆數：', len(lst_track_yes), s_yesterday)
# df_track_yes.set_index('id', inplace=True)
# df_track_close_yes = df_track_yes[['close']]
# df_track_close_yes.columns = ['close_yesterday']

# # 昨天外資、投信同步買進，只保留隔天有上漲的股票
# df_track_today = df_today[df_today.index.isin(lst_track_yes)].copy()
# df_track_today['increase'] = 0
# df_track_today['mean20'] = 0
# df_track_today = df_track_today.join(df_track_close_yes)
# df_track_today['increase'] = df_track_today['close'] - df_track_today['close_yesterday']
# df_track_today = df_track_today[df_track_today['increase']>=0]
# df_track_today['mean20'] = df_track_today.index.to_series().apply(cal_mean_20)
# # <月線2%內
# df_track_today = df_track_today[df_track_today['close'] < df_track_today['mean20'] * 1.02]
# lst_increase = list(df_track_today.index)
# print('隔天上漲且 <月線2%內筆數：', len(lst_increase), lst_increase)

# # join (今日股價、今日法人買賣超)
# df_track_today_inv = df_track_today.join(df_investor)
# df_track_today_inv.insert(1, 'date', s_today)
# df_track_today_inv

# # # save file
# df_track_yes = df_track_yes[df_track_yes.index.isin(lst_increase)]
# df_track_updated = pd.concat([df_track_yes.iloc[:,:-1],df_track_today_inv])
# df_track_updated.reset_index(inplace=True)
# df_track_updated.sort_values(['id', 'date'], ascending=[True, False], inplace=True)
# df_track_updated.to_excel(path_track_yesterday, index=False)
# df_track_updated

# 今天外資、投信同步買超

In [10]:
path_track_today = path_investor + 'investors_2days_{}.xlsx'.format(s_today)

df_inv_price = df_today[df_today.index.isin(lst_investor)].copy()
df_inv_price['increase'] = 0
df_inv_price['mean20'] = 0

df_inv_today = df_inv_price.join(df_investor)
df_inv_today

print('今天外資投信同步買超筆數：', len(lst_investor), lst_investor)

# save file
df_inv_today.insert(1, 'date', s_today)
df_inv_today.reset_index(inplace=True)
df_inv_today.to_excel(path_track_today, index=False)
df_inv_today.head(2)

今天外資投信同步買超筆數： 34 ['3481', '2409', '2303', '2330', '3034', '0050', '4919', '2353', '2634', '2454', '2379', '3704', '6533', '2301', '3017', '2376', '6719', '1476', '2885', '6285', '2327', '4770', '2356', '6515', '6781', '2748', '3661', '4961', '4904', '2404', '1590', '6830', '6592A', '6282']


,id,name,date,amount,open,high,low,close,increase,mean20,investor,trust,dealer_T,dealer_F
0,0050,元大台灣50,20230208,9239,120.1,120.95,120.0,120.85,0,0,1476,100,208,3249
1,1476,儒鴻,20230208,1309,496.0,515.00,496.0,513.00,0,0,636,201,-24,18


## 外資、投信同步買超 (檢查交易量突破2倍、收盤價突破10天盤整)

In [11]:
## 今日符合篩選條件，新增至追蹤
# 漲幅 > 1.5%，K棒 > 1%
df_add = df_today.loc[lst_investor,:]
df_add.sort_index(inplace=True)

df_add['rise%'] = round((df_add['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_add['K%'] = round( (df_add['close'] - df_add['open'])/df_add['open']*100, 2 )    # K棒%
df_add = df_add[ (df_add['rise%']>1.5) & (df_add['K%']>1) ]
df_add_min = df_add.copy()
df_add_min_yes = df_add.copy()

df_add = check_10days(df_add)
lst_add_ori = list(df_add.index)
print('今日新增:',len(lst_add_ori),'筆\n', lst_add_ori)

lst_add = df_add.index
df_add = df_add.loc[lst_add,:]
df_add

2379, 收盤價：364.5, 10日內最高價：352.0, 布林通道差：52.76, 布林帶寬：0.16
2409, 收盤價：17.7, 10日內最高價：17.1, 布林通道差：2.03, 布林帶寬：0.12
3034, 收盤價：390.0, 10日內最高價：366.5, 布林通道差：44.92, 布林帶寬：0.13
3481, 收盤價：13.05, 10日內最高價：12.55, 布林通道差：1.63, 布林帶寬：0.14
6781, 收盤價：854.0, 10日內最高價：810.0, 布林通道差：145.76, 布林帶寬：0.19
今日新增: 5 筆
 ['2379', '2409', '3034', '3481', '6781']


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,
2379,瑞昱,7063,340.0,364.50,339.0,364.50,9.13,7.21,9.0
2409,友達,112562,17.1,17.90,16.9,17.70,4.12,3.51,7.0
3034,聯詠,20690,374.5,398.00,373.5,390.00,7.14,4.14,9.0
3481,群創,200166,12.6,13.25,12.5,13.05,4.40,3.57,7.0
6781,AES-KY,2702,789.0,854.00,787.0,854.00,9.91,8.24,7.0


## 量價前30名 (檢查交易量突破2倍、收盤價突破10天盤整)

In [12]:
print('量價前30名:', lst_amt)
df_amt = check_10days(df_amt)
lst_amt = list(df_amt.index)
print('量價前30名，10日盤整:',len(lst_amt),'筆', lst_amt)
df_amt

量價前30名: ['2330', '3443', '3034', '3661', '3035', '2454', '1795', '2303', '4763', '6533', '6531', '8996', '6235', '4919', '3481', '2379', '6781', '6719', '2049', '6415', '2409', '3454', '1760', '5269', '6515', '2425', '3711', '0050', '8016', '00878']
3034, 收盤價：390.0, 10日內最高價：366.5, 布林通道差：44.92, 布林帶寬：0.13
3481, 收盤價：13.05, 10日內最高價：12.55, 布林通道差：1.63, 布林帶寬：0.14
2379, 收盤價：364.5, 10日內最高價：352.0, 布林通道差：52.76, 布林帶寬：0.16
6781, 收盤價：854.0, 10日內最高價：810.0, 布林通道差：145.76, 布林帶寬：0.19
2409, 收盤價：17.7, 10日內最高價：17.1, 布林通道差：2.03, 布林帶寬：0.12
量價前30名，10日盤整: 5 筆 ['3034', '3481', '2379', '6781', '2409']


,name,amount,open,high,low,close,value_amt,check
id,,,,,,,,
3034,聯詠,20690,374.5,398.00,373.5,390.00,80.0,9.0
3481,群創,200166,12.6,13.25,12.5,13.05,26.0,7.0
2379,瑞昱,7063,340.0,364.50,339.0,364.50,25.0,9.0
6781,AES-KY,2702,789.0,854.00,787.0,854.00,23.0,7.0
2409,友達,112562,17.1,17.90,16.9,17.70,19.0,7.0


## 外資 or 投信 買超

In [13]:
## 外資 or 投信 買超
df_inv_t = df_today.loc[lst_investor_trust,:]
df_inv_t.sort_index(inplace=True)

df_inv_t['rise%'] = round((df_inv_t['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_inv_t['K%'] = round( (df_inv_t['close'] - df_inv_t['open'])/df_inv_t['open']*100, 2 )    # K棒%
df_inv_t = df_inv_t[ (df_inv_t['rise%']>1) & (df_inv_t['K%']>1) ]
df_inv_min = df_inv_t.copy()
df_inv_min_yes = df_inv_t.copy()

df_inv_t = check_10days(df_inv_t)
lst_inv_t = list(df_inv_t.index)
print('今日新增:',len(lst_inv_t),'筆\n', lst_inv_t)

df_inv_t

00891, 收盤價：13.15, 10日內最高價：13.06, 布林通道差：1.85, 布林帶寬：0.15
00892, 收盤價：12.03, 10日內最高價：11.87, 布林通道差：1.77, 布林帶寬：0.16
查無資料 Hist < 10 1
查無資料 Hist < 10 1
- 031543.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
- 032413.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 084342.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 089123.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
1409, 收盤價：18.6, 10日內最高價：18.25, 布林通道差：0.98, 布林帶寬：0.05
2108, 收盤價：41.95, 10日內最高價：41.6, 布林通道差：3.76, 布林帶寬：0.09
2379, 收盤價：364.5, 10日內最高價：352.0, 布林通道差：52.76, 布林帶寬：0.16
2409, 收盤價：17.7, 10日內最高價：17.1, 布林通道差：2.03, 布林帶寬：0.12
3029, 收盤價：44.95, 10日內最高價：44.6, 布林通道差：4.52, 布林帶寬：0.11
3034, 收盤價：390.0, 10日內最高價：366.5, 布林通道差：44.92, 布林帶寬：0.13
3481, 收盤價：13.05, 10日內最高價：12.55, 布林通道差：1.63, 布林帶寬：0.14
3592, 收盤價：385.0, 10日內最高價：363.0, 布林通道差：50.04, 布林帶寬：0.14
6142, 收盤價：10.9, 10日內最高價：10.55, 布林通道差：1.0, 布林帶寬：0.1
6215, 收盤價：26.6, 10日內最高價：24.8, 布林通道差：3.58, 布林帶寬：0.15
6781, 收盤價：854.0, 

,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,
00891,中信關鍵半導體,19136,12.87,13.17,12.87,13.15,3.30,2.18,9.0
00892,富邦台灣半導體,10126,11.80,12.08,11.80,12.03,3.35,1.95,7.0
1409,新纖,7455,18.25,18.65,18.15,18.60,2.20,1.92,9.0
2108,南帝,2342,40.45,42.00,40.40,41.95,3.84,3.71,9.0
2379,瑞昱,7063,340.00,364.50,339.00,364.50,9.13,7.21,9.0
2409,友達,112562,17.10,17.90,16.90,17.70,4.12,3.51,7.0
3029,零壹,1564,44.35,45.00,44.20,44.95,2.04,1.35,7.0
3034,聯詠,20690,374.50,398.00,373.50,390.00,7.14,4.14,9.0
3481,群創,200166,12.60,13.25,12.50,13.05,4.40,3.57,7.0


## 外資、投信同步買超 (檢查創新低、昨日最低點)

In [14]:
# 今日創新低
df_add_min = check_10days(df_add_min, check='min')
lst_add_min = list(df_add_min.index)
print('今日新增:',len(lst_add_min),'筆\n', lst_add_min)
df_add_min

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


In [15]:
# 昨日最低點
df_add_min_yes = check_10days(df_add_min_yes, check='yes')
lst_add_min_yes = list(df_add_min_yes.index)
print('今日新增:',len(lst_add_min_yes),'筆\n', lst_add_min_yes)
df_add_min_yes

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


## 外資 or 投信 買超(檢查創新低、昨日最低點)

In [16]:
df_inv_min = check_10days(df_inv_min, check='min')
lst_inv_min = list(df_inv_min.index)
print('今日新增:',len(lst_inv_min),'筆\n', lst_inv_min)

df_inv_min

查無資料 Hist < 10 1
查無資料 Hist < 10 1
- 031543.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
- 032413.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 084342.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
- 089123.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


In [17]:
df_inv_min_yes = df_inv_t.copy()
df_inv_min_yes = check_10days(df_inv_min_yes, check='yes')
lst_inv_min_yes = list(df_inv_min_yes.index)
print('今日新增:',len(lst_inv_min_yes),'筆\n', lst_inv_min_yes)

df_inv_min_yes

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


# K、D值

In [18]:
import os, argparse, datetime, logging, traceback, time
import time, random, warnings
import pandas as pd
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

def main(IdDaysDict, verbose=False):
    ### Output
    col = 'StockID, StockName, Date, KRaw, DRaw, JRaw, K, D, J'
    dfStockKDJ = pd.DataFrame(columns = col.split(', '))

    ### XpathSetting ###
    # XpathSettingButton ='/html/body/div[23]/input'
    # XpathIdxLengthInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[4]/td[2]/input'
    # XpathShiftInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td[2]/input'
    # XpathSettingConfirmButton = f'/html/body/div[24]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
    # XpathPreviousDayButton = '/html/body/div[20]/nobr/table/tbody/tr/td[2]/nobr/input[2]'
    XpathAnchor = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[2]/td/canvas'
    XpathStockName = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[1]/td/table/tbody/tr/td[2]/nobr/b'
    # XpathDate = '/html/body/div[3]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'                           # OldVersion.~20221122
    # XpathK = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[1]/font'
    # XpathD = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[2]/font'
    # XpathJ = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[3]/font'

    ### webdriver
    if verbose: print('Opening Webdirver...')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--start-maximized')
    # chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=chrome_options)
    driver.maximize_window()
    FirstSetting = True
    ErrorMessage = []
    progress = IdDaysDict.items() if verbose else tqdm(IdDaysDict.items())
    for STOCK_ID, DAY in progress:
        if verbose:print(f'Fetching  StockID  :  {STOCK_ID}  information ...\t(1/{DAY})\t', end = '')
        url = f'https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={STOCK_ID}&CHT_CAT2=DATE'
        driver.get(url)
        # OldVersion
            # settingButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'StockCanvas_btnConfig')))
        try:
            settingButton = driver.find_element(By.ID, 'StockCanvas_btnConfig')
            settingButton.click()
        except Exception as e:
            print(f'There is no data available for this StockID : {STOCK_ID}')
            ErrorMessage.append(e)
            continue

        if FirstSetting:
            idxLengthInput = driver.find_element(By.ID, 'CFG_InitIndexLength')
            shiftInput = driver.find_element(By.ID, 'CFG_CtrlShiftCnt')
            for n in range(20,25):
                try:
                    XpathSettingConfirmButton = f'/html/body/div[{n}]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
                    settingConfirmButton = driver.find_element(By.XPATH, XpathSettingConfirmButton)
                except:
                    pass
            idxLengthInput.send_keys('\b\b\b1')
            shiftInput.send_keys('\b\b1')
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))
            settingConfirmButton.click()
        else:
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))

        for i in range(DAY):
            stockName = driver.find_element(By.XPATH, XpathStockName).text.split()[1]
            
            for n in range(3,5):
                try:
                    XpathDate = f'/html/body/div[{n}]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'
                    date = driver.find_element(By.XPATH, XpathDate).text.strip()
                except:
                    pass
            
            for n in range(9,15):
                try:
                    XpathK = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[1]/font'
                    XpathD = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[2]/font'
                    XpathJ = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[3]/font'
                    KRaw, DRaw, JRaw = map(lambda x: driver.find_element(By.XPATH, x).text[3:], (XpathK, XpathD, XpathJ))
                except:
                    pass

            K, D, J = map(lambda x: float(x[:-1]), (KRaw, DRaw, JRaw))
            dfStockKDJ.loc[len(dfStockKDJ)] = tuple([STOCK_ID, stockName, date, KRaw, DRaw, JRaw, K, D, J])
            if verbose: print(f'|| {STOCK_ID} | {stockName} | {date} | K: {KRaw:^6s} | D: {DRaw:^6s} | J: {JRaw:^6s} ||')
            if i == DAY - 1: break # early break

            # change date
            if verbose: print(f"Fetching the previous day's information ...\t({i+2}/{DAY})\t", end = '')
            previousDayButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnCtrlShiftLeft')))
            previousDayButton.click()
            time.sleep(1)
        if verbose: print()
    driver.close()
    if verbose:
        print('All query is done!!')
        print('There is some Error occured, please check!')
        for e in ErrorMessage:
            print(e)
    return dfStockKDJ


In [19]:
df_track = pd.read_excel('../stock_track.xlsx')
lst_track = lst_inv_t + lst_add_min_yes + lst_add_min
lst_track += list(df_track['stock_id'])
print(len(lst_track))

dic_track = {str(i):1 for i in lst_track}
df_KD = main(dic_track, verbose=False)
display(df_KD)

70


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:02<00:00, 2.92MB/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [05:21<00:00,  4.66s/it]


,StockID,StockName,Date,KRaw,DRaw,JRaw,K,D,J
0,00891,中信關鍵半導體,2023/02/08 (三),87▲,87.1▼,86.8▲,87.0,87.1,86.8
1,00892,富邦台灣半導體,2023/02/08 (三),89.1▲,89.4▼,88.6▲,89.1,89.4,88.6
2,1409,新纖,2023/02/08 (三),87.4▲,82▲,98.3▲,87.4,82.0,98.3
3,2108,南帝,2023/02/08 (三),77.7▲,73▲,87▲,77.7,73.0,87.0
4,2379,瑞昱,2023/02/08 (三),76.6▲,73.5▲,82.8▲,76.6,73.5,82.8
...,...,...,...,...,...,...,...,...,...
64,8478,東哥遊艇,2023/02/08 (三),28.7▼,41▼,4▼,28.7,41.0,4.0
65,9802,鈺齊-KY,2023/02/08 (三),38.3▼,39.2▼,36.5▼,38.3,39.2,36.5
66,9910,豐泰,2023/02/08 (三),33.5▲,32.7▲,35▲,33.5,32.7,35.0
67,9921,巨大,2023/02/08 (三),34.6▲,37.8▼,28.2▲,34.6,37.8,28.2


In [20]:
df_KD[(df_KD['K']<30) & (df_KD['K']<df_KD['D'])].sort_values('K')

,StockID,StockName,Date,KRaw,DRaw,JRaw,K,D,J
64,8478,東哥遊艇,2023/02/08 (三),28.7▼,41▼,4▼,28.7,41.0,4.0


# 下載Gmail股票交易excel

In [21]:
from google_apis import create_service

CLIENT_FILE = 'client_secret.json'
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(CLIENT_FILE, API_NAME, API_VERSION, SCOPES)

def search_emails(query_string, label_ids=None):
    try:
        message_list_response = service.users().messages().list(
                userId='me',
                labelIds=label_ids,
                q=query_string
        ).execute()
        message_items = message_list_response.get('messages')
        next_page_token = message_list_response.get('nextPageToken')
        return message_items
    except Exception as e:
        print(e)
        
def get_file_data(message_id, attachment_id, file_name, save_locatioin):
    response = service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()
    
    file_data = base64.urlsafe_b64decode(response.get('data').encode('utf-8'))
    return file_data

def get_message_detail(message_id, msg_format='metadata', metadata_headers=None):
    message_detail = service.users().messages().get(
        userId='me',
        id=message_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

gmail v1 service created successfully


In [22]:
import os
import base64
from tqdm import tqdm
from datetime import datetime

def download_excel():
    query_string = 'has:attachment'
    email_messages = search_emails(query_string)
    save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
    this_month = s_today[:-2]
    
    last_month = (s_today[:4] + str(datetime.now().month - 1).zfill(2) if datetime.now().month > 1 else f"{datetime.now().year-1}12")
    if not os.path.exists(os.path.join(save_folder, this_month)):
        os.mkdir(os.path.join(save_folder, this_month))
    lst_excel = glob(os.path.join(save_folder, this_month, '交易明細*'))
    if len(lst_excel) > 0:
        latest_file = lst_excel[-1]
    else:
        latest_file = glob(os.path.join(save_folder, last_month, '交易明細*'))[-1]
    latest_file = os.path.basename(latest_file)
    print('local latest file', latest_file)

    for email_message in tqdm(email_messages):
        message_detail = get_message_detail(email_message['id'], msg_format='full', metadata_headers=['parts'])
        if '富果帳戶 交易明細' in message_detail['snippet'] and not 'Forwarded' in message_detail['snippet']:
            message_detail_payload = message_detail.get('payload')
            if 'parts' in message_detail_payload:
                for msg_payload in message_detail_payload['parts']:
                    file_name = msg_payload['filename']
                    body = msg_payload['body']
                    if 'attachmentId' in body:
                        folder = file_name[4:10]
                        if file_name == latest_file:
                            return
                        attachment_id = body['attachmentId']
                        attachment_content = get_file_data(email_message['id'], attachment_id, file_name, save_folder)
                        with open(os.path.join(save_folder, folder, file_name), 'wb') as f:
                            f.write(attachment_content)
                            print(f"File {file_name} is saved at {save_folder} {folder}")
download_excel()

local latest file 交易明細20230207.xlsx


  1%|▊                                                                                 | 1/100 [00:00<00:57,  1.74it/s]


In [23]:
df_profit = pd.DataFrame()
from glob import glob
save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
this_month = s_today[:-2]
for filename in glob(os.path.join(save_folder, '202211', '交易明細*')):
    df_profit = pd.concat([df_profit, pd.read_excel(filename)])
df_profit

C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Mia\.virtualenvs\stock-8IVDzipY\lib

,交易別,代碼,商品名稱,成交股數,成交單價,成交價金,手續費,交易稅,融資自備款,融資金額,融券擔保品,融券保證金,融券費,淨收付,交易日期
0,現買,2607,榮運,2000,26.75,53500,76,0,NaN,NaN,NaN,NaN,0,-53576,20221101
1,現買,6670,復盛應用,200,185.00,37000,52,0,NaN,NaN,NaN,NaN,0,-37052,20221101
0,現買,2345,智邦,150,241.00,36150,51,0,NaN,NaN,NaN,NaN,0,-36201,20221102
1,現買,6670,復盛應用,150,187.50,28124,40,0,NaN,NaN,NaN,NaN,0,-28164,20221102
2,現買,9802,鈺齊-KY,300,141.50,42450,60,0,NaN,NaN,NaN,NaN,0,-42510,20221102
0,現賣,8478,東哥遊艇,200,242.00,48400,68,145,NaN,NaN,NaN,NaN,0,48187,20221103
1,現買,9802,鈺齊-KY,100,139.50,13950,19,0,NaN,NaN,NaN,NaN,0,-13969,20221103
2,現買,9941,裕融,200,138.00,27600,39,0,NaN,NaN,NaN,NaN,0,-27639,20221103
0,現賣,2345,智邦,500,246.00,123000,175,369,NaN,NaN,NaN,NaN,0,122456,20221110
1,現賣,9802,鈺齊-KY,500,136.00,68000,96,204,NaN,NaN,NaN,NaN,0,67700,20221110


In [24]:
df_profit['成交股數'] = df_profit.apply(lambda row: row['成交股數']*(-1) if row['交易別']=='現賣' else row['成交股數'], axis=1)
df_profit[df_profit['交易別'].isin(['現買','現賣'])].groupby(['代碼','商品名稱']).sum()

C:\Users\Mia\AppData\Local\Temp\ipykernel_13868\536218408.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_profit[df_profit['交易別'].isin(['現買','現賣'])].groupby(['代碼','商品名稱']).sum()


,,成交股數,成交單價,成交價金,手續費,交易稅,融資自備款,融資金額,融券擔保品,融券保證金,融券費,淨收付,交易日期
代碼,商品名稱,,,,,,,,,,,,
2059,川湖,-176,1654.50,176512,249,376,0.0,0.0,0.0,0.0,0,73695,80884462
2345,智邦,-350,487.00,159150,226,369,0.0,0.0,0.0,0.0,0,86255,40442212
2457,飛宏,0,79.75,159500,226,254,0.0,0.0,0.0,0.0,0,9820,40442243
2607,榮運,0,54.00,108000,153,163,0.0,0.0,0.0,0.0,0,684,40442219
6285,啟碁,600,82.30,49380,70,0,0.0,0.0,0.0,0.0,0,-49450,20221117
6670,復盛應用,350,372.50,65124,92,0,0.0,0.0,0.0,0.0,0,-65216,40442203
8478,東哥遊艇,-200,242.00,48400,68,145,0.0,0.0,0.0,0.0,0,48187,20221103
9802,鈺齊-KY,200,569.50,170149,240,204,0.0,0.0,0.0,0.0,0,-34593,80884431
9941,裕融,200,138.00,27600,39,0,0.0,0.0,0.0,0.0,0,-27639,20221103


# 上市上櫃 各類股 資金流向
**找出資金流向幅度增長最高的類股**

In [25]:
# 上市、上櫃資金流向表
url_flow = 'https://fubon-ebrokerdj.fbs.com.tw/z/zb/zba/zba.djhtm'
tables = pd.read_html(url_flow)
df = tables[-1]
df.head(3)

,0,1,2,3,4,5,6,7
0,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08,上市資金流向表 最後更新時間: 02/08
1,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率
2,水泥,0.26%,食品,0.26%,塑膠,0.48%,紡織纖維,0.75%


In [26]:
def get_stock_type_format(df):
    df_all = pd.DataFrame()
    for i in range(1, len(df.columns)//2 + 1):
        i_end = i * 2
        df_tmp = df.iloc[:, i_end-2:i_end]
        df_all = pd.concat([df_all, df_tmp])
    df_all.columns = ['type_name', 'flow_rate']
    df_all.dropna(inplace=True)
    df_all.reset_index(drop=True, inplace=True)
    df_all['flow_rate'] = df_all['flow_rate'].apply(lambda x: x.replace('%', ''))
    df_all['flow_rate'] = df_all['flow_rate'].astype(float)
    df_all = df_all.sort_values('flow_rate', ascending=False)
    
    df_all_t = df_all.T
    df_all_t.columns = df_all_t.iloc[0,:]
    df_all_t = df_all_t.iloc[1:,:]
    df_all_t.insert(0, 'date', value=s_today)
#     df_all_t.to_excel('stock_type.xlsx', index=False)
    return df_all_t

def get_flow_diff(df_new, df_old):
    df_new = df_new.iloc[:, 1:]
    df_new.reset_index(inplace=True, drop=True)
    
    df_yes = df_old.iloc[[-1], :]
    df_yes = df_yes.iloc[:, 1:]
    df_yes.reset_index(inplace=True, drop=True)
    
    df_diff = df_new.subtract(df_yes, fill_value=0)
    df_diff.insert(0, 'date', 'Difference')
    return df_diff

def cal_type_flow(path_file, df_new):
    print(path_file)
    df = pd.read_excel(path_flow + path_file, index_col=None, engine='openpyxl')
    df_ori = df.iloc[:-1, :]
    df_diff = get_flow_diff(df_new, df_ori)
    df_type = pd.concat([df_ori, df_new])
    df_type = pd.concat([df_type, df_diff])
    df_type.to_excel(path_flow + path_file, index=False)
    return df_type.reset_index(drop=True)

In [27]:
df_type = df.iloc[2:10, :]
df_type.columns = df.iloc[1,:]

df_type_new = get_stock_type_format(df_type)

cal_type_flow('stock_type.xlsx', df_type_new).iloc[-3:,:]

stock_type.xlsx


,date,電子,半導體,航運業,電子零組件,化學生技醫療,金融保險,化工,電腦及週邊設備,光電,...,紡織纖維,電子通路,食品,水泥,觀光,橡膠,油電燃氣,造紙,資訊服務,玻璃陶瓷
120,20230207,56.39,29.16,3.4,8.23,14.88,3.07,3.31,6.1,4.59,...,1.07,0.33,0.41,0.32,1.33,0.21,0.09,0.1,0.11,0.09
121,20230208,67.97,42.61,3.13,7.47,7.22,2.31,2.6,6.67,4.64,...,0.75,0.29,0.26,0.26,0.82,0.17,0.06,0.06,0.22,0.06
122,Difference,11.58,13.45,-0.27,-0.76,-7.66,-0.76,-0.71,0.57,0.05,...,-0.32,-0.04,-0.15,-0.06,-0.51,-0.04,-0.03,-0.04,0.11,-0.03


In [28]:
df_otc = df.iloc[12:17, :]
df_otc.columns = df.iloc[11,:]

df_otc_new = get_stock_type_format(df_otc)

cal_type_flow('stock_type_otc.xlsx', df_otc_new).iloc[-3:, :]

stock_type_otc.xlsx


,date,電子,半導體,電子零組件,生技醫療,光電,通信網路,其他電子,電腦及週邊設備,電機機械,化工,文化創意,其他,觀光,資訊服務,鋼鐵,電子通路,航運業,建材營造,紡織纖維
120,20230207,62.35,31.73,5.89,27.73,5.92,8.05,4.36,4.82,1.47,1.61,2.63,1.37,0.74,0.85,0.75,0.73,0.07,0.11,0.02
121,20230208,61.94,34.74,4.99,28.68,5.91,8.05,2.61,4.36,1.72,1.43,2.48,1.23,0.33,0.7,0.55,0.59,0.05,0.11,0.02
122,Difference,-0.41,3.01,-0.9,0.95,-0.01,0.0,-1.75,-0.46,0.25,-0.18,-0.15,-0.14,-0.41,-0.15,-0.2,-0.14,-0.02,0.0,0.0


# 畫日K線圖、週K線圖，儲存每天股票資料

In [29]:
file_name = root + 'stock_20210825_1203.json'
with open(file_name) as f:
    dic_stock = json.load(f)

latest_date_file = datetime.strptime(sorted(dic_stock)[-1], '%Y-%m-%d').date()
print('目前記錄日期至:',latest_date_file, '新增前:', len(dic_stock), '筆')

目前記錄日期至: 2023-02-07 新增前: 326 筆


In [30]:

import time
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    print(str(date).split(' ')[0].replace('-',''))
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret


data_stock = {}
date_today = d_today
n_days = (d_today - latest_date_file).days
fail_count = 0
allow_continuous_fail_count = 5
print('須補天數:', n_days)
while len(data_stock) < n_days:
    print('parsing', date_today)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        if date_today.weekday()  in [5,6]:
            print('Weekend!')
            date_today -= datetime.timedelta(days=1)
            continue
        data_stock[date_today] = crawl_price(date_today)
        if not str(date_today) in dic_stock.keys():
            dic_stock[str(date_today)] = data_stock[date_today].to_dict()
            print("add to dict:", str(date_today))
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
        break
    
    # 減一天
    date_today -= datetime.timedelta(days=1)
    time.sleep(10)

print('新增後:', len(dic_stock))
with open(file_name, 'w') as f:
    json.dump(dic_stock, f)

須補天數: 1
parsing 2023-02-08
20230208
add to dict: 2023-02-08
success!
新增後: 327


# K、D值

In [31]:
# def plot_daily_weekly_K(lst_stock_id, dic_stock):
#     df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
#     df_close.index = pd.to_datetime(df_close.index)

#     df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
#     df_open.index = pd.to_datetime(df_open.index)

#     df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
#     df_high.index = pd.to_datetime(df_high.index)

#     df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
#     df_low.index = pd.to_datetime(df_low.index)

#     df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
#     df_volume.index = pd.to_datetime(df_volume.index)
    
#     # =========================== 畫K線圖 =================================================
#     for stock_id in tqdm(lst_stock_id):
#         thisyear = '2022'
#         print(stock_id)
#         if not stock_id in df_close.keys():
#             print(stock_id, '查無資料')
#             continue
#     #     stock_id = '2603'
#         dic_a_stock = {
#             'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#         }

#         # Daily DataFrame
#         df_stock = pd.DataFrame(dic_a_stock)
#         df_stock.insert(0,'stock_id', stock_id)
#         df_stock.sort_index(inplace=True)

#         fig_daily = plot_K_chart(df_stock, 'daily')

#         # Weekly DataFrame
#         period_type = '1W'
#         df_week = df_stock.resample(period_type).last()
#         df_week['close'] = df_stock['close'].resample(period_type).last()
#         df_week['open'] = df_stock['open'].resample(period_type).first()
#         df_week['high'] = df_stock['high'].resample(period_type).max()
#         df_week['low'] = df_stock['low'].resample(period_type).min()
#         df_week['volume'] = df_stock['volume'].resample(period_type).sum()

#         fig_weekly = plot_K_chart(df_week, 'weekly')

In [32]:
# df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
# df_close.index = pd.to_datetime(df_close.index)

# df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
# df_open.index = pd.to_datetime(df_open.index)

# df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
# df_high.index = pd.to_datetime(df_high.index)

# df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
# df_low.index = pd.to_datetime(df_low.index)

# df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
# df_volume.index = pd.to_datetime(df_volume.index)

# # =========================== 畫K線圖 =================================================
# stock_id = '2603'
# thisyear = '2022'
# print(stock_id)
# if not stock_id in df_close.keys():
#     print(stock_id, '查無資料')
# #     stock_id = '2603'
# dic_a_stock = {
#     'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
# }

# # Daily DataFrame
# df_stock = pd.DataFrame(dic_a_stock)
# df_stock.insert(0,'stock_id', stock_id)
# df_stock.sort_index(inplace=True)
# df_stock

In [33]:
# print('今日新增 && 在量價前30名', list(set(lst_add_ori)& set(lst_amt)) )
# print('昨天外資投信同步買超筆數 & 今天上漲：', len(lst_increase), '筆')
# plot_daily_weekly_K(lst_increase, dic_stock)

In [34]:
# print('今日觀察：', len(lst_add_ori), '筆')
# plot_daily_weekly_K(lst_add_ori, dic_stock)

In [35]:
# print('量價前30名：', len(lst_amt),'筆\n', lst_amt)
# lst_amt_without_add = [i for i in lst_amt if i not in lst_add_ori]
# plot_daily_weekly_K(lst_amt_without_add, dic_stock)

# 預測EPS

In [36]:
# def get_EPS(lst_stock_id, filename):
#     df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
#     df_quarterly_all = get_df_quarterly_all()
#     df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

#     # 取得公司發行股數
#     path_company_stock_num = root + 'company_list.xlsx'
#     df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
#     df_company_stock_num.set_index('stock_id', inplace=True)
    
#     lst_eps = []
#     for i in range(len(lst_stock_id)):
#         stock_id = lst_stock_id[i]
#         t_wait = np.random.randint(5,15)
#         print(stock_id, ', wait ', t_wait)
#         df = get_df_quarterly(stock_id)

#         if df is None:
#             print('df is None')
#             continue
#         if df.loc[df.index[0], '毛利率(%)'] < 20:
#             print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
#             continue
#         elif df.loc[df.index[0], '營益率(%)'] < 8:
#             print(stock_id, '：營益率' , df.loc[df.index[0], '營益率(%)'] , ' < 8%，跳過')
#             continue

#         lst_eps.append(stock_id)
#         # 計算歷年同季的營收成長率、EPS
#         df = set_growth_eps(df, df_today, stock_id)

#         # 將年度EPS寫到歷年excel裡
#         lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
#         min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
#         lst_year = [y for y in lst_year if int(y) > min_year]
#         lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

#         df.reset_index(inplace=True)
#         df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

#         stock_name = ""
#         if stock_id in df_today.index:
#             stock_name = df_today.loc[stock_id, 'name']
#             df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
#             df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

#         df.insert(0, 'name', value=stock_name)
#         df.insert(0, 'stock_id', value=stock_id)

#         df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
#         time.sleep(t_wait)

#         # ===============================
#         # 預測EPS ( check_EPS )
#         # ===============================
#         df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
#         df_concat = pd.concat([df, df_new_quarter])
#         df_concat.sort_values('quarter', ascending=False, inplace=True)
#         df_estimate = df_estimate.append(df_concat)

#     #預測EPS ( check_EPS )
#     df_estimate.to_excel(path_check+'estimate_{}_{}.xlsx'.format(filename, s_today), index=False)
#     return lst_eps